In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
data1 = pd.read_excel("C:/Users/samba/OneDrive/Desktop/TAMU CORPUS/Driscoll/combined_datasets/average_cc_0.xlsx")
data1

,day,average_cc_19_0,average_cc_20_0,average_cc_21_0
0,0,0.000200,0.000748,0.002520
1,13,0.007000,0.135708,0.473314
2,25,0.318998,4.658331,14.547168
3,35,12.320046,19.580000,21.776037
4,41,30.107956,23.749794,29.348252
5,49,43.503645,35.934409,46.459822
6,55,51.541058,42.831603,62.859898
7,62,64.439337,49.560600,76.352083
8,69,72.712199,56.289597,89.949687
9,77,82.363871,63.979879,97.915300


In [4]:
data2 = data1['average_cc_19_0']

In [5]:
data2

0      0.000200
1      0.007000
2      0.318998
3     12.320046
4     30.107956
5     43.503645
6     51.541058
7     64.439337
8     72.712199
9     82.363871
10    92.015543
11    96.820000
12    96.779071
13    96.364815
14    94.650000
Name: average_cc_19_0, dtype: float64

In [6]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

import os
#os.environ['TF_CPP_MIN_LOG_LEVEL']

In [7]:
from math import sqrt
from multiprocessing import cpu_count
from warnings import catch_warnings, filterwarnings

import numpy as np
from joblib import Parallel, delayed
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

In [8]:
def sarima_forecast(history, config):
    order, sorder, trend = config

    history = np.array(history)
    model = SARIMAX(
        history,
        seasonal_order=sorder,
        order=order,
        trend=trend,
        enforce_stationarity=False,
        enforce_invertibility=False
    ).fit(disp=False)
# forecast
    yhat = model.predict(len(history), len(history))
    return yhat[0]

In [9]:
# train, test split
n_test = 4
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]


# RMSE
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [10]:
# Exponential Smoothing parameter configuration
def sarima_configs(seasonal=[0]):
    models = list()
    # configuratin permuations
    p_params = [0, 1, 2]
    d_params = [0, 1]
    q_params = [0, 1, 2]
    t_params = ['n', 'c', 't', 'ct']
    P_params = [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2]
    m_params = seasonal
    # create configuration permutation
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
    return models

In [11]:
cfg_list = sarima_configs(seasonal=[0,15])
print(cfg_list[0])

[(0, 0, 0), (0, 0, 0, 0), 'n']


In [12]:
# test
res = sarima_forecast(data2, cfg_list[0])
data = data2.values

In [13]:
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    train, test = train_test_split(data, n_test)
    history = [x for x in train]
    for i in range(len(test)):
        yhat = sarima_forecast(history, cfg)
        predictions.append(yhat)
        history.append(test[i])
    error = measure_rmse(test, predictions)
    return error

In [14]:
error = walk_forward_validation(data, 1, cfg_list[0])
print(error)

94.65


In [15]:
def score_model(data, n_test, cfg, debug=False):
    result = None
    key = str(cfg)
    
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
        try:
            with catch_warnings():
                filterwarnings('ignore')
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    if result is not None:
        print('>Model[%s] %.3f' % (key, result))
    return (key, result)

In [16]:
res = score_model(data, 1, cfg_list[1])

>Model[[(0, 0, 0), (0, 0, 0, 15), 'n']] 94.650


In [17]:
def grid_search(data, cfg_list, n_test, parallel=True):
    scores = None
    if parallel:
        executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
        tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    scores = [r for r in scores if [1] != None]
    scores.sort(key=lambda tup:tup[1])
    return scores

In [ ]:
scores = grid_search(data, cfg_list[1], n_test)

In [ ]:
from pprint import pprint
pprint(scores)